# So you want to convert Tensorflow to MxNet, eh? 

Lets try parsing the API's and generate a mapping between the two. This might take a while

In [ ]:
import requests
from bs4 import BeautifulSoup

def soupify(url):
    return BeautifulSoup(requests.get(url).text, 'html.parser')

tf_api = {}
for a in soupify("https://www.tensorflow.org/api_docs/python/").find_all('a', href=True):
    if 'api_docs/python/tf' in a['href']:
        tf_api[a.text] = soupify(a['href']).find('article', attrs={'class':'devsite-article-inner'}).text

print(tf_api)

In [ ]:
mx_api = {}
for a in soupify("https://mxnet.incubator.apache.org/api/python/index.html").find_all('a', href=True):
    if 'api/python/' in a['href'] and a['href'].split('#')[0] not in mx_api:
        for dl in soupify(a['href']).find_all('dl', attrs={'class':'method'}):
            mx_api[dl.find('dt')['id']] = dl.text

print(mx_api)